In [1]:
import os
import pandas as pd
import numpy as np
import random
import json

In [2]:
read_dir='E:/文档/数据集/Data Sets used in Computing Education/ASSISTment Data Sets/ASSISTment Data/ASSISTments2017/anonymized_full_release_competition_dataset.csv'
save_dir='../data/a2017/'
test_ratio=0.2
prob_count_limit=15 #每个学生做过的习题下限

In [3]:
data_set=pd.read_csv(read_dir)

# 只取学生id、题目id、技能、对错，这几列
# 同一个学生同一道题，只保留最后一条答题记录
need_data=data_set.loc[:,['studentId','problemId','skill','correct']]\
    .drop_duplicates(subset=['studentId','problemId','skill'],keep='first').dropna(axis=0,how='any')
#删除没有技能对应的答题记录
need_data=need_data[need_data['skill']!='noskill']

D:\anaconda\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# 统计每个学生做了多少道题
problem_counter=need_data.groupby(by='studentId').count()

filtered_stu_boundary=prob_count_limit #学生做的题超过15道才算数

filtered_stu_id=problem_counter[problem_counter['problemId']>filtered_stu_boundary].index.to_numpy()

print('有效的学生数：',len(filtered_stu_id))

有效的学生数： 1678


In [5]:
def is_filtered_stu_id(x):
    if x in filtered_stu_id:
        return True
    else:
        return False

filtered_id=need_data['studentId'].apply(is_filtered_stu_id)
filtered_data=need_data[filtered_id]

In [6]:
# problem=item
# skill=knowledge
# id=code

problem_and_skill=filtered_data.loc[:,['problemId','skill']].dropna(axis=0,how='any').drop_duplicates()

skill_name=list(problem_and_skill['skill'].drop_duplicates())
skill_name_id=dict(zip(skill_name,range(1,len(skill_name)+1)))

with open(save_dir+'dict_knowledge_code.json','w') as f:
    json.dump(skill_name_id,f)

filtered_stu_id_old_new=dict(zip(filtered_stu_id.astype('str'),range(1,len(filtered_stu_id)+1)))
with open(save_dir+'dict_user_id.json','w') as f:
    json.dump(filtered_stu_id_old_new,f)
 
filtered_problem_id=np.array(list(set(need_data['problemId'])))
filtered_problem_id_old_new=dict(zip(filtered_problem_id.astype('str'),range(1,len(filtered_problem_id)+1)))
with open(save_dir+'dict_item_id.json','w') as f:
    json.dump(filtered_problem_id_old_new,f)

In [7]:
filtered_data['studentId']=filtered_data['studentId'].astype('str').map(filtered_stu_id_old_new)
filtered_data['problemId']=filtered_data['problemId'].astype('str').map(filtered_problem_id_old_new)
filtered_data['skill']=filtered_data['skill'].map(skill_name_id)

C:\Users\qtl\AppData\Local\Temp/ipykernel_5000/1646276698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['studentId']=filtered_data['studentId'].astype('str').map(filtered_stu_id_old_new)
C:\Users\qtl\AppData\Local\Temp/ipykernel_5000/1646276698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['problemId']=filtered_data['problemId'].astype('str').map(filtered_problem_id_old_new)
C:\Users\qtl\AppData\Local\Temp/ipykernel_5000/1646276698.py:3: SettingWithCopyWarning: 
A val

In [8]:
print(filtered_data.max())

studentId    1678
problemId    2210
skill         101
correct         1
dtype: int64


## step 2 生成item (pd.DataFrame(columns=[item_id，knowledge_code]))

In [9]:
problem_and_skill=filtered_data.loc[:,['problemId','skill']].dropna(axis=0,how='any').drop_duplicates()
reindex_data=problem_and_skill.set_index('problemId')

item_data=pd.DataFrame(columns=['item_id','knowledge_code'])
counter=0
for problem in range(1,len(filtered_problem_id)+1):
    item_data.loc[counter,'item_id']=str(problem)
    item_data.loc[counter,'knowledge_code']=reindex_data.loc[[problem],'skill'].tolist()
    counter+=1
item_data.to_csv(save_dir+'item.csv',index=False)

## step3 划分数据集为，train、valid、test

In [10]:
# columns=['user_id','item_id','score']

reindex_data=filtered_data.loc[:,['studentId','problemId','correct']].set_index('studentId')

train_data=[[],[],[]]
valid_data=[[],[],[]]
test_data=[[],[],[]]

for stu in range(1,len(filtered_stu_id)+1):
    stu_data=reindex_data.loc[stu,:]
    stu_problem=np.array(stu_data['problemId'])
    stu_correct=np.array(stu_data['correct'])

    length=len(stu_problem)
    index_list=list(range(length))
    # 划分测试集
    test_index = random.sample(index_list, int(length*test_ratio))
    temp_index = list(set(index_list)-set(test_index))
    # 划分训练集和验证集
    valid_index = random.sample(temp_index, int(len(test_index)*test_ratio))
    train_index = list(set(temp_index)-set(valid_index))

    train_data[0].extend([stu]*len(train_index))
    train_data[1].extend(stu_problem[train_index])
    train_data[2].extend(stu_correct[train_index])

    valid_data[0].extend([stu]*len(valid_index))
    valid_data[1].extend(stu_problem[valid_index])
    valid_data[2].extend(stu_correct[valid_index])

    test_data[0].extend([stu]*len(test_index))
    test_data[1].extend(stu_problem[test_index])
    test_data[2].extend(stu_correct[test_index])

In [11]:
train=pd.DataFrame({'user_id':train_data[0],'item_id':train_data[1],'score':train_data[2]})
valid=pd.DataFrame({'user_id':valid_data[0],'item_id':valid_data[1],'score':valid_data[2]})
test=pd.DataFrame({'user_id':test_data[0],'item_id':test_data[1],'score':test_data[2]})

In [12]:
train.to_csv(save_dir+'train.csv',index=False)
valid.to_csv(save_dir+'valid.csv',index=False)
test.to_csv(save_dir+'test.csv',index=False)